In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing required libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split,DataLoader,TensorDataset

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Reading the data you are provided with

In [ ]:
train=pd.read_csv("../input/digit-recognizer/train.csv")
test=pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
test.head()

In [ ]:
train.head()

## Inspecting the datasets

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
train.shape

In [ ]:
test.shape

## Defining predictors and target variable

In [ ]:
# Target variable
labels=train.label


In [ ]:
# Independent variables
train_ds=train.drop('label',axis=1)

In [ ]:
train_ds.head()

## Let's visualise the image and take a look at the label 

In [ ]:
plt.imshow(train_ds[0:1].values.reshape(28,28))
plt.axis("off")
print(labels[0])

## Converting all kinds of data to numpy arrays

In [ ]:
train_ds=train_ds.values
test_ds=test.values
labels=labels.to_numpy()             #labels was actually a pandas series


## Converting all kinds of data to Tensors

In [ ]:
train_ds=torch.tensor(train_ds)
test_ds =torch.tensor(test_ds)
labels = torch.tensor(labels)

# Organising data into Tensordataset
 ***Each sample will be retrieved by indexing tensors along the first dimension.***

In [ ]:
traiin_ds=TensorDataset(train_ds,labels)

In [ ]:
traiin_ds[0:2]

## Splitting the Tensordataset into train and validation data

In [ ]:
train_ds , val_ds = random_split(traiin_ds,(32000,10000))

In [ ]:
# Define hyperparameters
batch_size = 150
learning_rate = 0.001

## Create batches

In [ ]:
train_loader=DataLoader(train_ds , batch_size, shuffle = True)
val_loader=DataLoader(val_ds , batch_size, shuffle = False)


## Constructing the model

In [ ]:
# for each image
input_size = 784
num_class = 10



In [ ]:
class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(nn.Linear(28*28 , 512),
                                    nn.ReLU(),
                                    nn.Linear(512 , 100),
                                    nn.ReLU(),
                                    nn.Linear(100 , 10))
        
        
    def forward(self,x):
        out = self.linear(x)
        return out

model = MnistModel()

## Define optimizer

In [ ]:
opt      = torch.optim.Adam
opt = opt(model.parameters(),lr=learning_rate)

## Training the model using logistic regression

In [ ]:
def fit(epochs,model,data):
    
    loss_fun = F.cross_entropy
    hist = []
    for Epoch in range(epochs):
        for img,label in data:
            out = model(img/255)
            loss = loss_fun(out,label)
            loss.backward()
            opt.step()
            opt.zero_grad()
            hist.append(loss)
        
        if (Epoch+1)%10 ==0:
            print(f"Epoch:[{Epoch+1}/{epochs}] ; Loss : {loss}")
    return hist
      

In [ ]:
hist=fit(90,model,train_loader)

In [ ]:
def acc(data):
    accuracy=[]
    for img , label in data:
        out = model(img/255)
        _,pred_index =torch.max(out,dim=1)
        x=torch.sum(pred_index==label)
        x=x/len(label)
        x=x*100
        accuracy.append(x)
    return np.mean(accuracy)
        
        

In [ ]:
acc(val_loader)

In [ ]:
# plt.plot(hist, range(32100))

In [ ]:
submission= pd.read_csv("../input/digit-recognizer/sample_submission.csv")
submission.head()

In [ ]:
len(submission)


In [ ]:
def prediction(data):
    LABEL=[]
    
    out = model(data/255)
    out = F.softmax(out)     
    _,pred_index =torch.max(out,dim=1)
    LABEL.append(pred_index)   
    return LABEL

In [ ]:
x=prediction(test_ds)
len(x[0])

In [ ]:
x=x[0].numpy()

In [ ]:
submission["Label"]=x

In [ ]:
submission.to_csv("submission.csv",index=False)

## DO UPVOTE !!